# Processing Gowalla Checkins

This notebook processes the [original Gowalla checkins data](https://snap.stanford.edu/data/loc-gowalla.html) into a database-style format, with a few normalised tables (augmented with some freely-available data).

It represents the first step in a wider project, to create a pipeline from a PostgreSQL database to BigQuery, that automatically triggers updates and displays some simple dashboards in Looker.

---

## Setup

Import packages and other basic setup

In [2]:
import pandas as pd
from os import path

In [3]:
# path to raw Gowalla data
RAW_DATA = path.join("raw", "loc-gowalla_totalCheckins.txt.gz")

---

## Read data from file

Pandas has no issues reading the data from a .gz archive, so can just load the file directly. However, a couple of gotchas:
- There are no headers in the file, just the raw data;
- The delimiters used are tabs (i.e. `\t`), not commas.

In [5]:
df = pd.read_csv(
    RAW_DATA,
    header=None,
    names=["user", "check_in_time", "latitude", "longitude", "location_id"],
    delimiter="\t"
)
df.head()

,user,check_in_time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


That's what we're after :-)